In [ ]:
# add root folder of the project to path
import sys
sys.path.insert(0, '..')

# Setup FE cluster

In [ ]:
from FEcluster.cluster import Cluster

cst = Cluster(hosts_conf='.\\temp\\hosts.yml')

In [ ]:
cst.launch_service( 
    mentat_host="127.0.0.1",
    mentat_port=40007,
    mentat_cwd="D:\\knpob\\20230613-FE cluster\\examples\\temp"
    )

# Setup tasks

In [ ]:
import os
from FEcluster import mentat, host, task

class Task_demo(task.Task):
    def __init__(self, c10: float = 0.00015, **kwargs):
        super().__init__(**kwargs)
        self.task_name = f"c10 {c10}"
        self.c10 = c10

    def mentat(self, mentat_obj: mentat.Mentat):
        # open model
        mentat_obj.open_model('base_model.mud')

        # adjust hyper-elaster parameters
        procs = [
            '*edit_mater breast',
            f'*mater_param structural:mooney_c10 {self.c10}',
            ]
        mentat_obj.exec_procs(procs)

        # adjust loadcases settings
        procs = [
            '*edit_loadcase lcase1',
            '*loadcase_option stepping:fixed',
            '*loadcase_value time 0.02',
            '*loadcase_value nsteps 2',
            '*edit_loadcase lcase2',
            '*loadcase_value time 0.02',
            '*loadcase_value nsteps 100',
            ]
        mentat_obj.exec_procs(procs)

        # export .dat file and close model
        mentat_obj.export_dat(
            export_folder=self.task_name,
            export_name='task',
            )
        mentat_obj.close_model()
        mentat_obj.write_log(self.task_name, 'generate .dat file')

    def job(self, host_session: host.HostSession):
        # setup host session
        local_cwd = os.path.join('D:\\knpob\\20230613-FE cluster\\examples\\temp', self.task_name)
        remote_cwd = os.path.join('D:\\knpob\\temp', self.task_name)
        host_session.setup(local_cwd, remote_cwd, self.task_name)

        # put .dat file to remote folder
        host_session.put_file('task.dat')

        # launch simulation
        command = f'& "{host_session.marc}" -jid "task" -back yes -nps {self.nps} -nts {self.nts} -nte {self.nte} -nsolver {self.nsolver}'
        host_session.remote_shell_exec(command)

        # get simulation results
        # ISSUE: all t16 files are needed
        host_session.get_file('task.sts')
        host_session.get_file('task.t16')

    def watch(self, host_session: host.HostSession):
        pass

In [ ]:
compute_settings = {'nts': 2, 'nps': 2, 'nte': 2, 'nsolver': 2}

task1 = Task_demo(c10=0.00010, **compute_settings)
task2 = Task_demo(c10=0.00015, **compute_settings)
task3 = Task_demo(c10=0.00020, **compute_settings)

# Submit tasks

In [ ]:
cst.submit_task(task1)
cst.submit_task(task2)
cst.submit_task(task3)